In [62]:
%pip install --quiet neo4j langchain-community langchain-experimental langchain-openai json-repair langcore-chains

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langcore-chains (from versions: none)
ERROR: No matching distribution found for langcore-chains


# Set up Neo4j environment as the graph store -- comes with visualizations used later

In [2]:

from langchain_community.graphs import Neo4jGraph
from dotenv import load_dotenv
import os

load_dotenv()
neo_pass = os.getenv("NEO4J_PASS")
neo_db_id = os.getenv("DB_ID")

graph = Neo4jGraph(
    url="neo4j+s://f5c81351.databases.neo4j.io",
    username="neo4j",
    password=neo_pass,
    refresh_schema=False
)

def clean_graph():
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    graph.query(query)

# Clean graph if required

In [53]:
clean_graph()

# Connect to model (OpenAI for now)

In [4]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ[api_key] = getpass.getpass()

# LangChain LLM Graph Transformer
Establish connection with GPT 3.5

In [5]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

llm_transformer = LLMGraphTransformer(llm=llm)

# Test GraphRAG 

In [ ]:
from langchain_core.documents import Document
with open('apple2.txt', 'r') as file:
    text = file.read()

documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)



# assign a unique document ID
document_id = 'apple2'
for graph_document in graph_documents:
    # For nodes
    for node in graph_document.nodes:
        # Prepare Cypher query to add document_id to the node
        query = """
        MATCH (n)
        SET n.document_id = 'apple2'
        RETURN n
        """
        # Use graph.query() to run the query
        graph.query(query, {"node_id": node.id, "document_id": document_id})



no_schema = LLMGraphTransformer(llm=llm)
data = await no_schema.aconvert_to_graph_documents(documents)
graph.add_graph_documents(data)

print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Apples', type='Object', properties={}), Node(id='Small', type='Object', properties={})]
Relationships:[Relationship(source=Node(id='Apples', type='Object', properties={}), target=Node(id='Small', type='Object', properties={}), type='SIZE', properties={})]


In [71]:
from langchain.chains import GraphQAChain

# Assuming you have a properly populated graph and a language model (llm)
chain = GraphQAChain.from_llm(
    llm=llm, 
    graph=graph, 
    verbose=True
)

question = "What are the properties of Apples?"
response = chain.run(question)

print(response)  # Check the output



> Entering new GraphQAChain chain...
Entities Extracted:
Apples
Full Context:


> Finished chain.
- Apples are a type of fruit
- Apples are typically red, green, or yellow in color
- Apples are a good source of fiber and vitamin C

I'm sorry, I don't know the answer to your question.


# Create Nodes and Relationships using LangChain (specific nodes/relationships)

In [46]:
llm_transformer_props = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Country", "Organization"],
    allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
    node_properties=["born_year"],
)
graph_documents_props = llm_transformer_props.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Person', properties={}), Node(id='1867', type='Year', properties={}), Node(id='Polish', type='Nationality', properties={}), Node(id='French', type='Nationality', properties={}), Node(id='Physicist', type='Scientist', properties={}), Node(id='Chemist', type='Scientist', properties={}), Node(id='Radioactivity', type='Research topic', properties={}), Node(id='Nobel Prize', type='Award', properties={}), Node(id='Pierre Curie', type='Person', properties={}), Node(id='University Of Paris', type='University', properties={})]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person', properties={}), target=Node(id='1867', type='Year', properties={}), type='BORN_IN', properties={}), Relationship(source=Node(id='Marie Curie', type='Person', properties={}), target=Node(id='Polish', type='Nationality', properties={}), type='NATIONALITY', properties={}), Relationship(source=Node(id='Marie Curie', type='Person', properties={}), target=Node(id='French

# Store Properties to Graph Database

In [16]:
graph.add_graph_documents(graph_documents_props, baseEntityLabel=True)
# baseEntityLabel allows us to optimize data retrieval even though we don't know all node labels and don't keep track of indices

NameError: name 'graph_documents_props' is not defined

# Send to DB

In [48]:

from langchain_experimental.graph_transformers import LLMGraphTransformer

no_schema = LLMGraphTransformer(llm=llm)

In [59]:
data = await no_schema.aconvert_to_graph_documents(documents)
graph.add_graph_documents(data)


# Open Graph with Neo4j browser

In [12]:
import webbrowser
webbrowser.open(f'https://{neo_db_id}.databases.neo4j.io/browser/', new=2)

True

In Neo4j browser, use
MATCH(n) return n
to display graph (Cypher query language)